In [0]:
from google.colab import drive
from pathlib import Path
import tensorflow as tf
import keras
import os


if tf.test.is_gpu_available():
  print('Using GPU')
else:
  print('Not using GPU')

Using GPU


### Connect to data

In [0]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [0]:

root_path = Path('gdrive/My Drive/ADL/dataset')
print(os.listdir(root_path))
classes = ['smoking','not_smoking']

train_path = root_path/'train'
dirs = [ aDir.name for aDir in train_path.glob('*') if aDir.is_dir() ]
print(dirs)

['train', 'test', 'validation']
['not_smoking', 'smoking']


In [0]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array

In [0]:
data_generator = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range = 0.4,
                                   height_shift_range = 0.4,
                                   zoom_range=0.3,
                                   rotation_range=20,
                                   )

image_size = 224
batch_size = 32
train_generator = data_generator.flow_from_directory(
        root_path/'train',
        target_size=(image_size, image_size),
        batch_size=batch_size,
        class_mode='categorical')

num_classes = len(train_generator.class_indices)

Found 8244 images belonging to 2 classes.


In [0]:
keras.applications.ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)


Instructions for updating:
Colocations handled automatically by placer.
102858752/102853048 [==============================] - 9s 0us/step


In [0]:
def resnet50_model(num_classes=2):
  model = Sequential()
  #keras.applications.resnet.ResNet50(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
  model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
  model.add(Flatten())
  model.add(BatchNormalization())
  model.add(Dense(2048, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(1024, activation='relu'))
  model.add(BatchNormalization())
  model.add(Dense(num_classes, activation='softmax'))
  model.layers[0].trainable = False
  return model


model = resnet50_model(num_classes=num_classes)

In [0]:
#model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','error'])


In [0]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
from keras.models import Sequential 
from keras.layers import Dense, Activation 
output_dim = nb_classes = 2
input_dim = 2048


# model = Sequential() 
# model.add(Dense(output_dim, input_dim=input_dim, activation='softmax')) 
# batch_size = 128 
# nb_epoch = 20


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

single_batch = next(iter(train_generator))
model.fit(
        single_batch[0],
        single_batch[1],
        steps_per_epoch=1, # int(count/batch_size) + 1,
#        batch_size=batch_size,
        epochs=10)

# model.fit_generator(
#         train_generator,
#         steps_per_epoch=1, # int(count/batch_size) + 1,
# #        batch_size=batch_size,
#         epochs=10)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 4.5014 - acc: 0.6250
Epoch 2/10
1/1 [==============================] - 0s 136ms/step - loss: 1.4952 - acc: 0.8750
Epoch 3/10
1/1 [==============================] - 0s 121ms/step - loss: 1.0104 - acc: 0.9375
Epoch 4/10
1/1 [==============================] - 0s 123ms/step - loss: 1.0178 - acc: 0.9375
Epoch 5/10
1/1 [==============================] - 0s 119ms/step - loss: 1.0097 - acc: 0.9375
Epoch 6/10
1/1 [==============================] - 0s 121ms/step - loss: 1.0083 - acc: 0.9375
Epoch 7/10
1/1 [==============================] - 0s 120ms/step - loss: 1.0081 - acc: 0.9375
Epoch 8/10
1/1 [==============================] - 0s 120ms/step - loss: 1.0062 - acc: 0.9375
Epoch 9/10
1/1 [==============================] - 0s 122ms/step - loss: 0.6213 - acc: 0.9375
Epoch 10/10
1/1 [==============================] - 0s 121ms/step - loss: 0.1811 - acc: 0.9688


In [0]:
count = sum([len(files) for r, d, files in os.walk("../input/flowers-recognition/flowers/flowers/")])

model.fit_generator(
        train_generator,
        steps_per_epoch=int(count/batch_size) + 1,
        epochs=10)

Epoch 1/10
1051/1051 [==============================] - 178s 170ms/step - loss: 0.6599
Epoch 2/10
1051/1051 [==============================] - 180s 171ms/step - loss: 0.6075
Epoch 3/10
 323/1051 [========>.....................] - ETA: 2:04 - loss: 0.5875

KeyboardInterrupt: ignored